# 第1单元: 训练你的第一个深度强化学习智能体🚀
在这份笔记中, 你将训练你的第一个着陆器智能体**正确登陆月球🌕并将其分享到社区, 并记得实验不同的配置**

❓如果你有任何问题, 请在discord的#study-group-unit1频道发帖 👉 https://discord.gg/aYka4Yhff9

🎮 环境: [LunarLander-v2](https://www.gymlibrary.ml/environments/box2d/lunar_lander/)

📚 强化学习库: [Stable-Baselines3](https://stable-baselines3.readthedocs.io/en/master/)

⬇️ 这是你将在几分钟内实现的目标的示例([原始视频下载链接](https://huggingface.co/ThomasSimonini/ppo-LunarLander-v2/resolve/main/replay.mp4)). ⬇️

In [1]:
%%html
<video autoplay controls><source src="./assets/replay.mp4" type="video/mp4"></video>

## 这份笔记的目标🏆
在这份笔记学习结束后, 你将:
* 能够使用环境库**Gym**.
* 能够使用深度强化学习库**Stable-Baselines3**.
* 能够通过精彩的回放和得分🔥**发布你训练的智能体到Hugging Face Hub**.

## 这份笔记来自深度强化学习课程
![深度强化学习课程.jpg](./assets/DeepReinforcementLearningCourse.jpg)

在这个免费课程中, 你将:
* 📖 研究深度强化学习的**理论和实践**.
* 🧑‍💻 学习**使用流行的深度强化学习库**, 例如Stable Baselines3, RL Baselines3 Zoo和RLlib.
* 🤖️ 在独特的环境中训练**智能体**.

还有更多的课程 📚 内容 👉 https://github.com/huggingface/deep-rl-class

保持进度的最佳方式是加入我们的Discord服务器与社区和我们进行交流. 👉🏻 https://discord.gg/aYka4Yhff9

## 先决条件 🏗
在深入研究笔记之前, 你需要:

🔲 📚 [阅读第1单元的README.](https://github.com/huggingface/deep-rl-class/blob/main/unit1/README.md)

🔲 📚 先学习**强化学习基础**(蒙特卡洛MC, 时序差分TD和奖励假设...) 👉 https://huggingface.co/blog/deep-rl-intro

🔲 📢 注册[我们的Discord服务器](https://discord.gg/aYka4Yhff9)**并在#introduce-yourself频道介绍自己** 🥳

🔲 🐕 你是Discord新手吗? 请查看我们的**discord 101以获得最佳实践** 👉 https://github.com/huggingface/deep-rl-class/blob/main/DISCORD.Md

## 一个深度强化学习的小回顾📚
![image.png](./assets/image.png)

有**两种**方法可以找到你的最优策略:
* 通过**训练直接训练你的策略**: 基于策略的方法.
* 通过**训练预期回报的价值函数**, 智能体将在每个状态下使用我们的函数定义我们的策略: 基于价值的方法.

* 最后, 我们谈到深度强化学习是因为**我们引入了深度神经网络去评估采取的动作(基于策略)或估算状态的价值(基于价值), 所以得名“深度”**.

# 让我们训练一个深度强化学习着陆器智能体来正确的着陆月球🌕并且上传到 Hugging Face Hub

### 第0步: 设置GPU 💪
* 为了**更快的训练智能体, 我们将使用GPU**, 选择`修改 > 笔记本设置`
![image.png](./assets/image1.png)
* `硬件加速器 > GPU`
![image.png](./assets/image2.png)

在笔记中, 我们需要生成一个回放视频. 因此在Colab(或你本地的jupyter)中, **我们需要一个虚拟屏幕能渲染环境**(记录视频帧).

下面的单元格将安装虚拟屏幕库并创建和运行虚拟屏幕. 🖥

In [2]:
!apt install gitlfs ffmpeg
# 如果你使用IDE(例如PyCharm或VS Code)将不需要这些步骤.
!apt install python-opengl xvfb 
!pip install pyvirtualdisplay

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

The operation couldn’t be completed. Unable to locate a Java Runtime.
Please visit http://www.java.com for information on installing Java.

Looking in indexes: https://mirrors.aliyun.com/pypi/simple/


In [3]:
# 创建虚拟屏幕.
from pyvirtualdisplay import Display

virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

FileNotFoundError: [Errno 2] No such file or directory: 'Xvfb'

### 第1步: 安装依赖项 🔽
第一步是安装多个依赖项:
* `gym`: 包含LunarLander-v2环境🌛
* `stable-baselines3`: 深度强化学习库.
* `huggingface_sb3`: Stable-baselines3的插件, 用于从Hugging Face Hub 🤗 上下载或者上传模型.

In [ ]:
!pip install gym pygame box2d-py  # 如果使用Apple M1 conda install box2d-py
!pip install stable-baselines3
!pip install huggingface_sb3
!pip install pyglet  # 如果你使用IDE, 则不需要这些步骤.

### 第2步: 导入包 📦
我们导入的另一个库是 huggingface_hub, **它能从Hub上下载或者上传预训练模型**.

Hugging Face Hub 🤗 是一个任何人都可以分享和探索模型和数据集的地方. 它有版本控制, 评估, 可视化和其他功能, 可以允许你简单地与他人协作.

你可以在这看到全部可用的深度强化学习模型. 👉 https://huggingface.co/models?pipeline_tag=reinforcement-learning&sort=downloads

In [ ]:
import gym

from huggingface_sb3 import load_from_hub, package_to_hub, push_to_hub
from huggingface_hub import notebook_login  # 需要登陆Hugging Face账户才能将模型上传到Hub.

from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.env_util import make_vec_env

### 第3步: 了解什么是Gym以及它是如何工作的? 🤖️
🏋️ 这个包含我们环境的库叫做Gym. 你将在**深度强化学习中使用很多Gym的环境**. 

Gym库提供了两个东西:
* 允许你**创建强化学习环境**的接口.
* **一组环境**(gym-control, atrai, box2D...).

让我们看一个例子, 但首先让我们记住什么是强化学习循环.

![image.png](./assets/image.png)

在每一步:
* 我们的智能体从**环境**接收**状态S0** -- 我们接收我们游戏(环境)的第一帧画面.
* 基于**状态S0**, 智能体采取**行动A0** -- 我们的智能体将向右移动.
* 从环境产生中**新状态S1** -- 新的一帧画面.
* 环境给智能体一些**奖罚R1** -- 我们没有死(奖励+1).

在Gym中:

1⃣️ 我们使用`gym.make()`创建环境

2⃣️ 我们使用`observation = env.reset()`重置环境到初始状态

对于每一步:

3⃣️ 我们使用模型获取一个动作(在我们的例子中, 我们获取一个随机的动作)

4⃣️ 我们使用`env.step(action)`在环境中执行这个动作, 然后得到:
* `obsevation`: 新状态(st+1)
* `reward`: 我们执行动作后得到的奖罚
* `done`: 提示这局游戏是否结束
* `info`: 额外信息的字典(取决于具体的环境).

如果这局游戏结束:
* 我们再次使用`observation = env.reset()`重置环境到初始状态

**让我们看看这个例子!** 确保读懂代码.

In [ ]:
import gym

# 首先, 我们创建LunarLander-v2的环境.
env = gym.make('LunarLander-v2')  

# 重置环境到初始状态.
observation = env.reset()

for _ in range(20):
    # 如果你使用IDE, 可以取消渲染环境的注释.
    # env.render()
    # 获取一个随机动作.
    action = env.action_space.sample()  
    print('动作: ', action)
    
    # 执行动作, 获取新状态, 奖罚, 这局游戏是否结束和额外信息.
    observation, reward, done, info = env.step(action)
    
    # 如果这局游戏结束(这个例子中结束的标志是着陆, 坠毁或超时).
    if done:
        # 重置环境到初始状态.
        print('环境被重置.')
        observation = env.reset()

### 第4步: 创建LunarLander环境🌛并且了解它如何工作
#### [环境 🎮](https://www.gymlibrary.ml/environments/box2d/lunar_lander/)
在第一个教程中, 我们将训练一个[登月着陆器]((https://www.gymlibrary.ml/environments/box2d/lunar_lander/)智能体, 它将正确的登陆月球. 为此, 智能体需要**学习去调整它的速度和位置(水平, 垂直和角度)来正确着陆**.

---
💡 当你开始使用一个环境的好习惯是查看文档.

👉 https://www.gymlibrary.ml/environments/box2d/lunar_lander/

---

让我们一起看看环境是什么样的:

In [ ]:
# 我们使用 gym.make('<环境的名称>') 创建环境.
env = gym.make('LunarLander-v2')
env.reset()
print('_' * 5 + '可观察的环境' + '_' * 5)
print()
print('可观察的环境向量的形状', env.observation_space.shape)
print('随机采样环境', env.observation_space.sample())  # 获得一个随机的可观察环境空间.

我们看到`可观察的环境空间的形状 (8,)`, 这指出可观察空间是一个大小为8的向量, 每个值是着陆器的不同信息:
* 着陆器坐标(x)
* 着陆器坐标(y)
* 水平速度(x)
* 垂直速度(y)
* 角度
* 角速度
* 左侧起落架是否着陆
* 右侧起落架是否着陆

In [ ]:
print('_' * 5 + '动作空间' + '_' * 5)
print()
print('动作的总数', env.action_space.n)
print('随机动作', env.action_space.sample())  # 获得一个随机的动作.

动作空间(智能体可用的可能动作集)是有4个可用动作的离散值 🎮:
* 不行动,
* 启动左方向的引擎,
* 启动主引擎,
* 启动右方向的引擎.

奖励函数(在每个时间步给予的奖励)💰:
* 从屏幕顶部移动到着陆台, 零速度是100~140分.
* 启动主引擎每帧画面 -0.3分.
* 每侧起落架着地+10分.
* 如果着陆器坠毁(额外-100分)或停止(+100分), 则本轮游戏结束.
* 如果你的智能体得到200分, 你的任务就结束了.

#### 向量化环境
* 我们创建一个由16个环境组成的向量化环境(堆叠多个独立环境在一个环境的方法), 这样**我们将有更多样化的体验在训练过程**.

In [ ]:
# 创建环境.
env = make_vec_env('LunarLander-v2', n_envs=16)

### 第5步: 创建一个模型 🤖️
* 现在我们研究了我们的环境并且理解了我们的问题: **通过控制左, 右方向和主引擎能够正确的将着陆器降落到着陆台上**. 让我们构建一个算法来解决这个问题. 🚀
* 为此, 我们将使用我们的第一个深度强化学习库, [Stable Baselines3(SB3)](https://stable-baselines3.readthedocs.io/en/master/).
* SB3是**一组PyTorch强化学习算法的可靠实现集合**.
---
💡 使用新的库的好习惯是先阅读文档: https://stable-baselines3.readthedocs.io/en/master/ 然后尝试一些教程.

---

![image.png](./assets/image3.png)

为了解决这个问题, 我们将使用SB3 **PPO**. [PPO(又名近端策略优化(Proximal Policy Optimization))是一种在本课程中你将学到的最先进的深度强化学习算法之一.](https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html#example%5D)

PPO是以下各项的组合:
* 基于价值的强化学习方法: 通过学习一个动作-价值函数来告诉我们在**给定的状态和动作下的最有价值的动作**.
* 基于策略的强化学习方法: 通过学习一个策略来给我们**动作的概率分布**.

Stable-Baselines3 易于设置:

1⃣️ 你**创建一个环境**(我们的例子中已经在上面完成)

2⃣️ 你**定义一个你想使用的模型并实例化它**`model = PPO('MlpPolicy')`

3⃣️ 你通过`model.learn()`**训练智能体**, 并定义训练的时间步

```
# 创建环境
env = gym.make('LunarLander-v2')

# 实例化智能体
model = PPO('MlpPolicy', env, verbose=1)
# 训练智能体
model.learn(total_timesteps=int(2e5))
```

In [ ]:
# TODO: 定义MlpPolicy PPO架构
# 我们使用多层感知机(MlpPolicy)是因为输入是一个向量, 
# 如果我们输入是帧, 那我们将使用CnnPolicy.
model = 

#### 答案

In [ ]:
# 答案
# 我们添加了一些参数来更快的训练
model = PPO(policy='MlpPolicy',
            env=env,
            n_steps=1024,
            batch_size=64,
            n_epochs=4,
            gamma=0.99,
            gae_lambda=0.98,
            ent_coef=0.01,
            verbose=1)

### 第6步: 训练PPO智能体 🏃
* 让我们训练智能体 500000 步, 不要忘记使用Colab的GPU. 这大概需要10分钟, 但如果你仅想尝试一下, 你可使用更少的时间步.
* 在训练期间, ☕️好好休息一下吧 🤗

In [ ]:
# TODO: 训练500000步

# TODO: 指定模型的文件名并保存模型.
model_name = ''

#### 答案

In [ ]:
# 答案
# 训练500000步
model.learn(total_timesteps=500000)
# 保存模型
model_name = 'ppo-LunarLander-v2'
model.save(model_name)

### 第7步: 评估智能体 📈
* 现在我们的登月着陆器智能体已经训练好了🚀, 我们需要**检查它的性能**.
* Stable-Baselines3 提供了一个方法`evaluate_policy`来进行评估.
* 要填写哪些部分, 你需要[查看文档](https://stable-baselines3.readthedocs.io/en/master/guide/examples.html#basic-usage-training-saving-loading)
* 下一步, 我们将了解**如何自动评估和分享你的智能体到排行榜竞争**, 现在让我们自己做

💡 当你评估你的智能体时, 你不应该使用训练环境, 而是创建一个评估环境.

In [ ]:
# TODO: 评估智能体
# 创建一个新的评估环境
eval_env = 

# 使用10个评估周期和确定性动作评估模型
mean_reward, std_reward = 

# 打印结果


#### 答案

In [ ]:
eval_env = gym.make('LunarLander-v2')
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f'mean_reward={mean_reward:.2f} +/- {std_reward}')

* 就我而言, 在训练1百万步后我得到的平均奖励是`200.20 +/- 20.80`, 这意味着我们的登月着陆器智能体已经准备好登陆月球了.🌛🥳

### 第8步(不涉及核心内容, 可选): 发布我们训练好的模型到 Hub 上 🔥
现在我们看到经过训练之后得到了很棒的结果, 我们可以通过一行代码发布我们训练的模型到hub🤗上.

📚 库文档 👉 https://github.com/huggingface/huggingface_sb3/tree/main#hugging-face--x-stable-baselines3-v20

这有一个模型卡的例子(使用Space Invaders):

![ModelCard.gif](./assets/ModelCard.gif)

通过使用`package_to_hub`, **你可以评估, 记录回放视频, 生成智能体的模型卡并把它发布到hub**.

---
package_to_hub 函数
---

看这边:
* 你可以**展示我们的作品** 🔥
* 你可以**可视化智能体的活动** 👀
* 你可以**与社区分享其他人也可以使用的智能体** 💾
* 你可以**访问排行榜🏆以查看你的智能体和你同学的智能体相比如何** 👉 https://huggingface.co/spaces/ThomasSimonini/Lunar-Lander-Leaderboard

为了能分享你的模型到社区, 有以下三个步骤需要做:

1⃣️ (如果没有完成)创建一个Hugging Face账户 ➡ https://huggingface.co/join

2⃣️ 登陆账户, 然后你需要保存一个Hugging Face的身份验证令牌(token).

* 创建一个新的具有**写入规则**的令牌(https://huggingface.co/settings/tokens)

![image.png](./assets/image4.png)

* 复制令牌
* 运行下面的脚本并输入令牌

In [ ]:
notebook_login()
!git config --global credential.helper store

如果你使用IDE, 也可在终端中使用以下命令:

In [ ]:
!huggingface-cli login

3⃣️ 我们现在准备好使用`package_to_hub()`发布我们训练的智能体到🤗 hub 🔥.

让我们填写`package_to_hub`函数:
* `model`: 我们训练的模型.
* `model_name`: 我们在`model.save()`中定义的模型的名称.
* `model_architectrue`: 我们使用的模型架构: 在我们的例子中是PPO.
* `env_id`: 环境的名称, 在我们的例子中是`LunarLander-v2`.
* `eval_env`: 定义一个评估环境.
* `repo_id`: 将创建/更新的Hugging Face Hub仓库的名称(repo_id={你的用户名/仓库名}).

💡 **一个好的名字是{用户名}/{模型架构}-{环境名称}**

* `commit_message`: 提交时的信息.

In [ ]:
import gym
from stable_baselines3.common.vec_env import DummyVecEnv

from huggingface_sb3 import package_to_hub

# 创建评估环境.
eval_env = DummyVecEnv([lambda: gym.make(env_id)])

# TODO: 填写环境的名称.
env_id = ''

# TODO: 填写我们使用的模型结构.
model_architecture = ''

# TODO: 填写仓库id.
# `repo_id`是Hugging Face Hub中模型存储库的id
# (repo_id = {organization}/{repo_name} 例子 ThomasSimonini/ppo-LunarLander-v2)
repo_id = ''

# TODO: 填写提交信息.
commit_message = ''

# 在将仓库推送到hub之前, 需要保存评估模型, 生成模型卡以及录制智能体的回放视频.
package_to_hub(model=model,  # 我们训练的模型.
               model_name=model_name,  # 我们训练模型的名称.
               model_architecture=model_architecture,  # 我们使用的模型架构: 在我们的例子中是PPO.
               env_id=env_id,  # 环境的名称.
               eval_env=eval_env,  # 评估环境.
               repo_id=repo_id,  # Hugging Face Hub仓库的名称(repo_id={你的用户名/仓库名} 例子 ThomasSimonini/ppo-LunarLander-v2).
               commit_message=commit_message)

#### 答案

In [ ]:
import gym
from stable_baselines3.common.vec_env import DummyVecEnv

from huggingface_sb3 import package_to_hub

# 创建评估环境.
eval_env = DummyVecEnv([lambda: gym.make(env_id)])

# 填写环境的名称.
env_id = 'LunarLander-v2'

# 填写我们使用的模型结构.
model_architecture = 'PPO'

# 填写仓库id.
# 请更改成你自己的id!
repo_id = 'ThomasSimonini/TEST2ppo-LunarLander-v2'

# 填写提交信息.
commit_message = 'Upload PPO LunarLander-v2 trained agent.'

package_to_hub(model=model,  # 我们训练的模型.
               model_name=model_name,  # 我们训练模型的名称.
               model_architecture=model_architecture,
               env_id=env_id,
               eval_env=eval_env,
               repo_id=repo_id,
               commit_message=commit_message)

恭喜🥳你刚刚训练并上传了你的第一个深度强化学习智能体. 上面的脚步应该显示了模型仓库的链接, 例如: https://huggingface.co/osanseviero/test_sb3. 当你访问这个链接时, 你可以:
* 在右侧你可以看到智能体的回放视频.
* 点击"Files and versions"以查看仓库中的全部文件.
* 点击"Use in stable-baselines3"以获取如何加载模型的代码.
* 得到描述模型的模型卡(文件`README.md`).

在底层, Hub使用基于git的仓库(即使你不知道什么是git也不用担心), 这意味着你可以在实验和提高你的智能体以后更新新版本的模型.

使用排行榜🏆比较你和同学的LunarLander-v2结果 👉 https://huggingface.co/spaces/ThomasSimonini/Lunar-Lander-Leaderboard

## 额外的挑战(可选) 🏆
最好的学习方式就是**自己进行尝试**! 如你所见, 当前的智能体还有做到最好. 作为第一个建议, 你可以训练更多的时间步. 比如1000000步, 我们可以看到更好的结果!

在[排行榜](https://huggingface.co/spaces/ThomasSimonini/Lunar-Lander-Leaderboard)中, 你将找到你的智能体的位置. 你想要获得第一吗?

以下是一些实现这个目标的想法:
* 训练更多的时间步
* 尝试不同的`PPO`超参数. 你可以在 https://stable-baselines3.readthedocs.io/en/master/modules/ppo.html#parameters 看到它们
* 翻阅[Stable-Baselines3的文档](https://stable-baselines3.readthedocs.io/en/master/modules/dqn.html)并尝试其他的模型, 比如DQN.
* **发布你训练的新模型**到Hub上 🔥

在[排行榜](https://huggingface.co/spaces/ThomasSimonini/Lunar-Lander-Leaderboard)上**比较你和同学的LunarLander-v2结果** 🏆 👉 https://huggingface.co/spaces/ThomasSimonini/Lunar-Lander-Leaderboard

登月对你来说太无聊了? 尝试**其他环境**, 为什么不试试CartPole-v1, MountainCar-v0 或者 CarRacing-v0? [使用gym文档](https://www.gymlibrary.ml/)查询它们如何工作. 玩得开心🎉.

---
祝贺你完成本章! 这才是最重要的, 这还有**一些额外的信息**.

如果你仍然对这些感到困惑...这是完全正常的! **这对我和所有学习强化学习的人都是一样的**.

在继续尝试其他挑战之前, **花一点时间真正的掌握这些内容**. 理解这些内容并打下基础是非常重要的.

当然, 在后续课程中, 我们将会继续使用并再次解释这些内容, 但**最好是在开始下一章之前完全掌握这些**.

### 这是专门为你打造的课程 👷🏿‍♀️

我们希望根据你的反馈提高和改进课程. 如果你有一些建议, 请打开GitHub仓库的issue: https://github.com/huggingface/deep-rl-class/issues

第二单元见! 🔥
## 不断学习, 不断精彩! 